Reference : https://arxiv.org/abs/1804.02767

Architecture: https://towardsdatascience.com/yolo-v3-object-detection-53fb7d3bfe6b 

Full algorithm explanations: https://www.youtube.com/watch?v=5e5pjeojznk&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF&index=25 

For learning the implementation (not this implementation though): https://blog.paperspace.com/how-to-implement-a-yolo-v3-object-detector-from-scratch-in-pytorch-part-4/ (all parts https://blog.paperspace.com/tag/series-yolo/)

In [ ]:
#@title imports { form-width: "20%" }
import os
import random
import numpy as np
import math
from time import time
import importlib
from tqdm import tqdm  #if you just import tqdm you will have to call it like tqdm.tqdm()
from abc import ABCMeta, abstractmethod
from collections import OrderedDict, Iterable, defaultdict
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, ToTensor, ToPILImage
from torchvision import datasets

In [ ]:
#@title download data { form-width: "20%" }


datasets.VOCDetection(root='/content', year='2007', image_set='train', download=True) 

# !wget --quiet http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar .
!tar -xf VOCtest_06-Nov-2007.tar
!rm VOCtest_06-Nov-2007.tar

#DO THIS! otherwise because of long path you will get FileNotFoundError
!cp -r /content/VOCdevkit/VOC2007/* /content/ 

#just for cleaning AND put this in separate cell otherwise it won't work
shutil.rmtree('/content/VOCdevkit')

In [ ]:
#@title write class names file { form-width: "20%" }

%%writefile voc_classes.txt
aeroplane
bicycle
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor

Writing voc_classes.txt


In [ ]:
#@title write color names file { form-width: "20%" }

a = np.array([[ 50,   0, 255],
       [127, 255,   0],
       [255, 229,   0],
       [  0, 102, 255],
       [255,   0, 229],
       [  0, 255, 178],
       [127,   0, 255],
       [  0,  25, 255],
       [204,   0, 255],
       [255,   0,   0],
       [ 51, 255,   0],
       [  0, 178, 255],
       [255,  76,   0],
       [255,   0,  76],
       [  0, 255, 255],
       [  0, 255, 102],
       [255, 153,   0],
       [203, 255,   0],
       [255,   0, 152],
       [  0, 255,  25]], dtype=np.int32) #float is also possible

# here each triplet describes RGB, red green blue, ranging from 0 to 255
# notice the number of colours corresponds to the number of classes
# notice one value is allways zero. This is just because we picked most intensive colours to be noticable
# you can play with it using colour picker
np.save('voc_colors.npy', a)

In [ ]:
#@title write cleaned training image names file for VOC { form-width: "20%" }

%%writefile ./ImageSets/Main/train.txt
000012
000017
000023
000026
000032
000033
000034
000035
000036
000042
000044
000047
000048
000061
000064
000066
000073
000077
000078
000083
000089
000091
000104
000112
000122
000129
000133
000134
000138
000140
000141
000147
000153
000154
000159
000161
000162
000163
000164
000171
000173
000174
000187
000189
000192
000193
000194
000198
000200
000207
000209
000219
000220
000222
000225
000228
000235
000242
000250
000256
000259
000262
000263
000276
000278
000282
000288
000294
000296
000306
000307
000311
000312
000317
000320
000325
000331
000334
000337
000344
000347
000349
000355
000359
000367
000370
000372
000379
000382
000387
000391
000394
000395
000400
000404
000406
000407
000411
000416
000430
000431
000438
000446
000450
000454
000463
000468
000469
000470
000474
000476
000477
000484
000489
000496
000503
000508
000516
000518
000519
000522
000524
000525
000526
000528
000535
000537
000541
000544
000549
000550
000552
000554
000555
000559
000565
000577
000583
000589
000590
000592
000597
000605
000609
000612
000620
000622
000625
000632
000633
000635
000648
000654
000657
000671
000672
000680
000685
000688
000689
000695
000699
000700
000709
000710
000711
000726
000729
000731
000733
000739
000740
000753
000754
000761
000764
000767
000768
000770
000774
000793
000796
000804
000805
000810
000812
000818
000820
000822
000823
000827
000828
000829
000830
000845
000849
000850
000851
000859
000860
000865
000867
000871
000887
000888
000889
000892
000898
000899
000900
000902
000904
000906
000908
000912
000915
000919
000929
000936
000943
000950
000951
000954
000958
000962
000964
000965
000966
000967
000977
000980
000987
000989
000991
000993
000996
000997
000999
001001
001002
001008
001010
001011
001014
001015
001024
001036
001043
001050
001057
001060
001064
001068
001071
001073
001077
001078
001079
001082
001101
001106
001112
001113
001119
001127
001129
001130
001140
001147
001151
001152
001156
001158
001168
001171
001172
001174
001182
001191
001194
001204
001205
001207
001209
001212
001214
001226
001229
001230
001234
001237
001239
001240
001248
001258
001263
001268
001269
001270
001273
001279
001287
001294
001299
001304
001309
001312
001314
001315
001323
001325
001326
001327
001332
001333
001334
001345
001346
001348
001364
001365
001378
001384
001385
001388
001390
001393
001395
001402
001404
001405
001406
001408
001409
001414
001418
001420
001421
001426
001427
001434
001436
001442
001450
001451
001453
001455
001457
001468
001470
001479
001480
001483
001485
001486
001488
001492
001494
001498
001499
001501
001504
001512
001515
001517
001521
001524
001526
001528
001529
001532
001539
001548
001555
001556
001557
001559
001563
001576
001579
001586
001590
001593
001594
001597
001604
001607
001610
001611
001612
001622
001630
001633
001636
001643
001649
001650
001651
001654
001661
001662
001669
001673
001676
001680
001683
001684
001688
001690
001699
001707
001708
001711
001713
001714
001717
001721
001723
001729
001732
001733
001734
001738
001739
001741
001750
001752
001754
001758
001759
001761
001765
001766
001768
001777
001780
001787
001789
001800
001806
001809
001810
001821
001825
001828
001832
001834
001836
001840
001841
001843
001845
001853
001854
001858
001861
001864
001870
001881
001892
001894
001896
001898
001902
001903
001904
001906
001915
001922
001928
001930
001937
001938
001941
001950
001952
001954
001960
001963
001971
001977
001978
001980
001981
001985
001989
001995
001999
002001
002002
002004
002006
002012
002015
002020
002025
002027
002034
002037
002039
002042
002043
002047
002049
002051
002055
002056
002061
002068
002069
002095
002096
002104
002108
002116
002117
002120
002126
002132
002134
002139
002151
002153
002155
002156
002158
002166
002170
002172
002176
002178
002179
002180
002182
002186
002187
002191
002192
002193
002194
002196
002197
002199
002208
002212
002215
002219
002221
002224
002234
002237
002238
002241
002247
002249
002253
002255
002256
002260
002265
002277
002279
002280
002284
002287
002291
002293
002306
002307
002310
002311
002315
002318
002320
002321
002323
002334
002335
002342
002347
002350
002354
002355
002359
002362
002368
002373
002384
002392
002401
002403
002405
002410
002411
002413
002419
002420
002423
002433
002436
002439
002442
002443
002445
002448
002458
002461
002465
002466
002468
002471
002472
002478
002480
002481
002483
002490
002494
002496
002500
002501
002502
002512
002514
002518
002519
002533
002534
002539
002544
002545
002547
002554
002555
002558
002559
002569
002571
002572
002579
002590
002594
002595
002599
002603
002609
002611
002625
002627
002634
002635
002645
002647
002648
002653
002662
002664
002666
002669
002680
002682
002683
002684
002691
002697
002702
002704
002710
002713
002715
002722
002730
002735
002737
002738
002744
002745
002749
002755
002757
002759
002763
002765
002766
002774
002778
002779
002782
002783
002791
002795
002796
002801
002804
002807
002816
002817
002820
002826
002834
002841
002844
002845
002848
002855
002858
002864
002866
002867
002868
002869
002870
002873
002881
002899
002906
002914
002919
002931
002934
002937
002939
002953
002956
002957
002958
002962
002969
002975
002976
002987
002988
002989
002990
002992
002995
003002
003003
003007
003011
003013
003024
003027
003034
003042
003047
003051
003053
003061
003063
003066
003074
003077
003083
003085
003088
003092
003100
003103
003105
003106
003107
003108
003110
003116
003122
003124
003133
003134
003135
003138
003140
003145
003146
003147
003149
003150
003155
003157
003159
003161
003163
003165
003169
003175
003181
003183
003184
003185
003188
003202
003204
003205
003211
003214
003229
003231
003233
003236
003240
003242
003244
003247
003253
003254
003259
003260
003261
003269
003270
003273
003279
003280
003282
003284
003290
003292
003303
003308
003320
003330
003331
003336
003337
003338
003339
003343
003349
003350
003354
003355
003356
003359
003363
003365
003367
003369
003373
003374
003379
003380
003382
003392
003395
003396
003401
003406
003408
003412
003413
003416
003417
003420
003421
003424
003430
003433
003436
003439
003441
003450
003452
003466
003477
003484
003487
003489
003491
003493
003496
003497
003499
003500
003506
003508
003509
003510
003511
003522
003524
003525
003529
003539
003548
003549
003550
003551
003555
003564
003565
003575
003576
003577
003585
003586
003588
003596
003599
003603
003604
003605
003608
003609
003614
003621
003622
003625
003627
003628
003629
003634
003635
003642
003644
003645
003646
003656
003658
003662
003663
003664
003671
003678
003679
003681
003688
003694
003695
003698
003699
003700
003704
003705
003713
003714
003732
003735
003740
003743
003748
003749
003751
003752
003758
003759
003763
003767
003773
003779
003781
003784
003786
003788
003790
003792
003797
003806
003807
003811
003817
003818
003824
003827
003828
003830
003834
003835
003847
003849
003856
003859
003860
003861
003865
003866
003874
003879
003887
003889
003890
003898
003899
003907
003912
003913
003921
003932
003935
003936
003939
003945
003949
003953
003956
003961
003969
003970
003971
003974
003983
003987
003988
003991
003993
003997
003998
004005
004008
004009
004013
004014
004016
004017
004019
004023
004028
004033
004034
004035
004037
004046
004052
004058
004067
004091
004093
004095
004100
004106
004111
004120
004121
004129
004131
004133
004136
004137
004138
004140
004146
004149
004152
004158
004163
004164
004168
004169
004170
004171
004189
004190
004196
004200
004209
004215
004220
004221
004223
004224
004228
004231
004232
004237
004241
004242
004244
004247
004253
004255
004256
004263
004269
004270
004271
004272
004273
004280
004281
004283
004287
004291
004292
004296
004300
004303
004307
004315
004318
004322
004325
004327
004333
004338
004339
004345
004347
004359
004360
004361
004365
004367
004370
004371
004372
004376
004379
004386
004387
004389
004391
004392
004404
004434
004436
004439
004441
004452
004470
004471
004479
004481
004484
004496
004500
004502
004508
004510
004514
004517
004519
004520
004524
004526
004537
004540
004544
004548
004549
004551
004553
004562
004563
004565
004566
004570
004571
004576
004579
004584
004587
004591
004595
004597
004604
004605
004607
004611
004612
004622
004623
004625
004627
004628
004631
004634
004636
004643
004644
004648
004651
004656
004671
004675
004679
004683
004685
004686
004687
004691
004693
004694
004701
004702
004705
004706
004710
004714
004715
004718
004723
004735
004737
004742
004743
004747
004748
004753
004754
004760
004773
004776
004779
004782
004783
004790
004792
004793
004794
004797
004799
004801
004808
004815
004823
004828
004830
004832
004836
004837
004841
004842
004846
004848
004849
004857
004869
004873
004876
004879
004882
004885
004897
004898
004902
004905
004907
004910
004911
004913
004929
004946
004951
004955
004958
004961
004962
004966
004968
004972
004973
004974
004976
004984
004987
004990
004991
004992
004995
005001
005004
005006
005007
005016
005018
005020
005023
005024
005026
005027
005029
005032
005033
005045
005047
005052
005057
005058
005061
005065
005068
005071
005073
005078
005084
005086
005090
005093
005094
005097
005101
005107
005108
005114
005121
005122
005124
005129
005130
005134
005138
005143
005153
005156
005168
005169
005171
005173
005177
005181
005183
005186
005189
005190
005191
005202
005203
005208
005215
005217
005219
005223
005231
005236
005244
005245
005246
005257
005258
005259
005260
005262
005269
005273
005283
005285
005288
005290
005292
005297
005303
005304
005307
005311
005318
005327
005336
005337
005338
005344
005345
005351
005358
005360
005363
005368
005369
005373
005374
005387
005388
005389
005391
005396
005404
005405
005406
005408
005410
005413
005414
005417
005420
005424
005433
005440
005445
005448
005450
005451
005453
005455
005457
005467
005478
005483
005487
005489
005496
005499
005508
005509
005511
005514
005515
005519
005524
005526
005527
005536
005541
005542
005544
005547
005563
005566
005568
005574
005579
005582
005585
005591
005592
005599
005600
005601
005603
005605
005609
005611
005624
005625
005630
005631
005636
005637
005639
005644
005648
005654
005658
005668
005669
005680
005686
005695
005697
005699
005700
005704
005705
005710
005713
005715
005718
005728
005730
005731
005735
005738
005740
005742
005752
005756
005757
005764
005765
005769
005780
005782
005783
005784
005786
005789
005803
005805
005806
005813
005814
005817
005821
005824
005826
005831
005836
005838
005840
005843
005850
005851
005859
005860
005861
005864
005867
005873
005881
005884
005885
005888
005889
005893
005895
005899
005901
005903
005905
005908
005909
005910
005911
005918
005920
005923
005930
005938
005947
005948
005951
005960
005961
005964
005971
005980
005983
005984
005990
005992
006004
006009
006011
006020
006023
006025
006030
006033
006038
006043
006061
006065
006066
006067
006070
006073
006074
006078
006079
006088
006091
006095
006096
006100
006103
006104
006105
006123
006128
006130
006131
006134
006135
006140
006141
006156
006158
006162
006166
006170
006171
006172
006174
006175
006176
006177
006179
006180
006181
006183
006187
006189
006196
006208
006210
006221
006223
006224
006225
006229
006230
006236
006238
006243
006247
006250
006251
006261
006262
006264
006267
006270
006272
006275
006279
006285
006289
006290
006291
006299
006304
006305
006320
006329
006341
006344
006349
006352
006353
006362
006363
006366
006367
006369
006371
006374
006375
006381
006382
006395
006400
006411
006417
006418
006419
006427
006429
006433
006434
006436
006438
006447
006448
006455
006458
006459
006462
006466
006470
006472
006474
006475
006476
006482
006483
006486
006495
006499
006501
006503
006506
006515
006523
006524
006536
006547
006548
006549
006550
006551
006556
006560
006564
006569
006595
006597
006602
006605
006609
006610
006612
006622
006626
006627
006635
006636
006637
006638
006648
006652
006654
006658
006660
006674
006684
006689
006694
006695
006697
006698
006703
006704
006706
006707
006708
006714
006726
006727
006731
006734
006735
006736
006738
006740
006748
006753
006755
006766
006773
006777
006781
006782
006784
006794
006805
006806
006810
006822
006824
006825
006833
006836
006839
006840
006844
006845
006847
006848
006849
006852
006858
006864
006866
006868
006869
006874
006883
006887
006893
006896
006899
006900
006909
006910
006911
006912
006914
006916
006917
006919
006930
006931
006939
006943
006947
006948
006950
006958
006959
006968
006971
006976
006983
007002
007003
007006
007007
007011
007016
007018
007023
007025
007029
007033
007036
007039
007040
007045
007050
007062
007064
007072
007073
007075
007078
007079
007080
007088
007089
007090
007092
007093
007095
007105
007108
007113
007121
007125
007128
007129
007130
007133
007138
007150
007152
007154
007159
007163
007166
007168
007177
007180
007182
007184
007185
007193
007194
007197
007205
007213
007214
007219
007222
007223
007234
007241
007243
007250
007256
007261
007263
007271
007279
007285
007289
007295
007298
007305
007308
007322
007323
007325
007327
007334
007336
007351
007361
007365
007369
007370
007373
007375
007381
007385
007389
007394
007396
007398
007410
007411
007413
007417
007419
007421
007425
007431
007437
007446
007454
007458
007466
007467
007468
007474
007477
007479
007481
007483
007490
007491
007493
007497
007503
007513
007519
007521
007524
007530
007535
007536
007538
007540
007544
007565
007566
007570
007572
007575
007578
007586
007590
007594
007600
007601
007606
007611
007619
007621
007629
007631
007633
007637
007653
007654
007655
007663
007667
007683
007685
007692
007696
007697
007699
007704
007713
007718
007721
007729
007731
007735
007736
007740
007748
007749
007751
007753
007762
007767
007775
007777
007781
007790
007791
007795
007803
007809
007810
007814
007819
007820
007821
007831
007836
007838
007840
007847
007853
007854
007859
007863
007864
007872
007876
007878
007883
007885
007898
007900
007901
007905
007908
007910
007911
007914
007915
007923
007925
007926
007932
007939
007940
007953
007959
007963
007964
007968
007974
007976
007980
007991
007996
008001
008004
008005
008008
008012
008017
008019
008026
008037
008040
008042
008043
008044
008049
008051
008053
008062
008063
008064
008067
008072
008075
008076
008079
008082
008083
008084
008093
008095
008096
008098
008106
008108
008116
008117
008121
008127
008130
008137
008139
008142
008150
008163
008164
008166
008169
008174
008186
008188
008197
008199
008202
008203
008204
008213
008216
008218
008223
008226
008232
008235
008248
008250
008252
008253
008254
008260
008261
008262
008263
008269
008272
008280
008282
008296
008301
008302
008310
008311
008312
008313
008315
008316
008317
008322
008332
008336
008338
008341
008342
008346
008351
008360
008372
008374
008381
008384
008385
008388
008391
008397
008398
008403
008409
008422
008425
008426
008427
008437
008442
008443
008445
008449
008452
008453
008456
008462
008465
008466
008467
008468
008470
008475
008477
008478
008482
008483
008495
008506
008517
008523
008529
008530
008533
008536
008549
008550
008558
008559
008568
008581
008585
008587
008588
008595
008596
008602
008610
008615
008617
008618
008628
008633
008645
008655
008663
008665
008670
008676
008688
008691
008699
008702
008706
008710
008720
008723
008725
008727
008731
008732
008738
008741
008744
008748
008750
008755
008756
008757
008760
008764
008768
008770
008771
008776
008783
008784
008790
008794
008806
008809
008811
008813
008814
008815
008819
008838
008840
008841
008847
008856
008862
008865
008872
008878
008879
008883
008885
008886
008891
008900
008905
008909
008920
008923
008926
008929
008930
008932
008933
008936
008939
008944
008948
008958
008960
008961
008962
008966
008967
008968
008969
008970
008971
008973
008975
008978
008979
008980
008985
008987
008988
008989
008995
008999
009000
009004
009005
009016
009018
009020
009027
009029
009032
009036
009042
009045
009049
009058
009059
009063
009066
009068
009073
009078
009080
009086
009098
009099
009100
009106
009108
009114
009117
009121
009123
009136
009144
009148
009153
009160
009161
009166
009173
009175
009181
009184
009185
009191
009196
009197
009200
009205
009208
009209
009214
009215
009218
009227
009230
009238
009242
009245
009251
009252
009255
009259
009269
009270
009271
009272
009283
009285
009287
009288
009289
009290
009295
009296
009299
009306
009307
009308
009316
009318
009324
009325
009327
009333
009336
009339
009342
009343
009358
009359
009362
009365
009377
009386
009388
009389
009392
009393
009394
009398
009406
009407
009409
009410
009411
009413
009417
009418
009419
009420
009421
009422
009424
009429
009432
009434
009446
009458
009460
009463
009465
009466
009469
009476
009488
009490
009491
009496
009497
009499
009504
009508
009512
009515
009516
009518
009520
009523
009524
009526
009528
009537
009541
009542
009545
009549
009551
009557
009562
009566
009573
009576
009577
009579
009584
009585
009587
009596
009600
009605
009609
009613
009614
009615
009618
009621
009623
009629
009634
009637
009638
009644
009650
009654
009656
009659
009664
009666
009668
009671
009679
009684
009691
009693
009702
009703
009707
009709
009713
009717
009718
009721
009729
009733
009734
009735
009749
009755
009756
009762
009763
009774
009776
009789
009790
009792
009797
009800
009805
009807
009808
009810
009813
009825
009828
009830
009832
009834
009839
009842
009845
009848
009851
009852
009855
009859
009860
009867
009868
009869
009872
009874
009877
009878
009879
009882
009884
009887
009896
009904
009911
009918
009920
009926
009938
009940
009942
009944
009945
009949
009959
009961

Overwriting ./ImageSets/Main/train.txt


In [ ]:
#@title data handling { form-width: "20%" }

class TransformAnnotation(object):
    def __init__(self, keep_difficult=False):
        self.keep_difficult = keep_difficult

    def __call__(self, target):
        res = []
        for obj in target.iter('object'):
            difficult = int(obj.find('difficult').text) == 1
            if not self.keep_difficult and difficult:
                continue
            name = obj[0].text.lower().strip()
            bbox = obj[4]
            bndbox = [int(bb.text)-1 for bb in bbox]
            res += [bndbox + [name]]
        return res
    
class VOCDataset(Dataset):
    def __init__(self, path, input_shape=(416, 416), mode='train', dataset='voc'):
        self.path = path
        self.mode = mode
        self.S = input_shape[0]
        self.transform = Compose([Resize(input_shape), ToTensor()])
        self.target_transform = TransformAnnotation()
        
        f_path = os.path.join(path, 'ImageSets/Main/train.txt')
        self.files = [i.strip('\n') for i in open(f_path).readlines()]
        random.seed(42)
        random.shuffle(self.files)
        
        self.ip = os.path.join(path, 'JPEGImages/')
        self.lp = os.path.join(path, 'Annotations/')
        
        self.classes = [c.strip() for c in open(f'{dataset}_classes.txt').readlines()]

    def __getitem__(self, i):
        f = self.files[i]
        x = Image.open(self.ip + f + '.jpg').convert('RGB')
        W, H = x.size
        x = self.transform(x)
        if self.mode=='test':
            return x
        else:
            y = ET.parse(self.lp + f + '.xml').getroot()
            y = self.target_transform(y)
            for yi in y:
                x1, y1, x2, y2, c = yi
                ci = self.classes.index(c)
                w, h = x2-x1, y2-y1
                cx, cy = x2-w/2, y2-h/2
                yi[:] = [cx, cy, w, h, ci]
            
            scale = torch.tensor([1/W, 1/H, 1/W, 1/H, 1])
            y = torch.tensor(y)*scale
            return x, y

    def __len__(self):
        return len(self.files)
    
    def classes(self,):
        return self.classes
    


In [ ]:
#@title calculators { form-width: "20%" }
 
def toTensor(image, size):
    image_ = image.resize(size)
    x = ToTensor()(image_).unsqueeze(0)
    return x

def IoUnion(boxA, boxB):
    Acx, Acy, Aw, Ah = boxA[:4]
    Bcx, Bcy, Bw, Bh = boxB[:4]
    
    x1 = max(Acx - Aw/2, Bcx - Bw/2)
    x2 = min(Acx + Aw/2, Bcx + Bw/2)
    y1 = max(Acy - Ah/2, Bcy - Bh/2)
    y2 = min(Acy + Ah/2, Bcy + Bh/2)

    w_cross = max(x2 - x1, 0)
    h_cross = max(y2 - y1, 0)

    Sa = Aw * Ah
    Sb = Bw * Bh
    intersection = w_cross * h_cross
    union = Sa + Sa - intersection
    iou = intersection/union
    return iou

def SoftIoUnion(boxesA, boxesB):

    Acx, Acy, Aw, Ah = boxesA[0], boxesA[1], boxesA[2], boxesA[3]
    Bcx, Bcy, Bw, Bh = boxesB[0], boxesB[1], boxesB[2], boxesB[3]
    
    x1 = torch.max(Acx - Aw/2, Bcx - Bw/2)
    x2 = torch.min(Acx + Aw/2, Bcx + Bw/2)
    y1 = torch.max(Acy - Ah/2, Bcy - Bh/2)
    y2 = torch.min(Acy + Ah/2, Bcy + Bh/2)

    W_ab = torch.max(x2 - x1, torch.zeros(len(x2)))
    H_ab = torch.max(y2 - y1, torch.zeros(len(y2)))
    
    Sa = Aw * Ah
    Sb = Bw * Bh
    intersection = W_ab * H_ab
    union = Sa + Sb - intersection
    return intersection/union
    
# be aware, that this particular NMS function will be used only for the pruning of predictions
# NMS involved in training is separately implemented as a part of the model below 
# be aware, in NMS, we find IoU between two predictions bounding boxes, instead of prediction vs ground truth 
# we iteratively take box(es) with highest objectnes score and eliminate all other boxes that overlap 
# with it more than 0.5, basically we prune the (nearest=0.5 IoU) neigbourhood. 
def NonMaxSuppression(boxes: torch.tensor, IoU:float =0.5):  # boxes: torch.tensor, thresh_iou : float
    if len(boxes) == 0:
        return boxes

    objectnesScores = [(1-b[4]) for b in boxes]    # b[4] contains the objectnes score for the box. 
    sorted_idx = np.argsort(objectnesScores)       # best scores are first as 1-b[4] gives lowest value for highest score as scores are between 0-1  
    preds = []

    for i in range(len(boxes)):
        box_i = boxes[sorted_idx[i]]               #takes the box with highest(modified to lowest as trick) score
        if objectnesScores[i] > -1:                # you will understand this in the end of the 2nd loop
            preds.append(box_i)                    #put the best scored box in a separate list that was empthy        
            for j in range(i+1, len(boxes)):       #loop in all following boxes
                if objectnesScores[j] > -1:        # you will understand this in the end of the 2nd loop
                    box_j = boxes[sorted_idx[j]]   #just take the following boxes to test if the 
                    if IoUnion(box_i, box_j) > IoU:  #IoU of following boxes vs best score box is more than 0.5
                        objectnesScores[j] = -1    #if IoU with non best scoring box is above 0.5 it's eliminated
    return preds

def YoloHead(preds):
    preds = np.array(preds)
    # Select Boxes, objectnesScores and Classes
    boxes = preds[:, :4]
    objectnesScores = preds[:, 4]
    classes = preds[:, -1].astype('int')
    # Boxes to Corners
    cx, cy, w, h = boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3]
    x1, y1 = cx - w/2, cy - h/2
    x2, y2 = cx + w/2, cy + h/2
    boxes = np.stack((x1, y1, x2, y2), axis=1)
    
    return objectnesScores, boxes, classes

def DrawBoxes(img, objectnesScores, boxes, classes, dataset='voc'):
    # Copy Image (to exclude changes in the original image)
    image = img.copy()
    W, H = image.size
    scale = np.array([W, H, W, H])
    thickness = (W + H)//300
    # Import Classes
    font = ImageFont.load_default() 
    class_names = [c.strip() for c in open(f'{dataset}_classes.txt').readlines()]
    colors = np.load(f'{dataset}_colors.npy') #not (same, allow_pickle=True)
    # Cycle through the Classes
    for i, c in reversed(list(enumerate(classes))):
        color = colors[c]
        cls = class_names[c]
        box = boxes[i]
        score = objectnesScores[i]
        # Box`s Label
        label = '{} {:.2f}'.format(cls, score)
        # Draw Image
        draw = ImageDraw.Draw(image)
        size = draw.textsize(label, font)
        # Choose Frontiers
        left, top, right, bottom = box *scale
        # Exclude out of the image situations
        left = max(0, int(left))
        top = max(0, int(top))
        right = min(W, int(right))
        bottom = min(H, int(bottom))

        if top - size[1] >= 0:
            origin = np.array([left, top - size[1]])
        else:
            origin = np.array([left, top + 1])
            
        # Draw Frame
        for k in range(thickness):
            draw.rectangle([left + k, top + k, right - k, bottom - k], outline=tuple(color)) #outline=tuple(color) if custom colors
        # Draw Label
        draw.rectangle([tuple(origin), tuple(origin + size)], fill=tuple(color)) #fill=tuple(color) if custom colors
        draw.text(tuple(origin), label, fill=(0, 0, 0), font=font)   
        del draw
    return image

def PictureDetection(model, img, size=(416,416), threshold=.25, IoU=.5, dataset='voc'):
    if type(img) is str:
        image = Image.open(img).convert('RGB')
        x = toTensor(image, size)
    else:
        image = ToPILImage()(img)
        x = ToTensor()(image.resize(size))
    preds = model.predict(x, threshold)[0]
    if len(preds) != 0:
        preds = NonMaxSuppression(preds, IoU)
        objectnesScores, boxes, classes = YoloHead(preds)
        image = DrawBoxes(image, objectnesScores, boxes, classes, dataset)
    return image

def VideoDetection(model, img, size=(416,416), threshold=.25, IoU=.5, dataset='voc'):
    image = ToPILImage()(img)
    x = ToTensor()(image.resize(size))
    preds = model.predict(x, threshold)[0]
    preds = NonMaxSuppression(preds, IoU)
    if len(preds) != 0:
        objectnesScores, boxes, classes = YoloHead(preds)
        image = DrawBoxes(image, objectnesScores, boxes, classes, dataset)
    return image

def PlotSample(x, y, dataset='voc'):
    x = ToPILImage()(x)
    y = y.numpy()
    W, H = x.size
    scale = np.array((W, H, W, H))
    thickness = (W + H)//300
    font = ImageFont.load_default() 
    classes = [c.strip() for c in open(f'{dataset}_classes.txt').readlines()]
    colors = np.load(f'{dataset}_colors.npy')

    for yi in y:
        draw = ImageDraw.Draw(x)
        cls=classes[int(yi[-1])]
        color=colors[int(yi[-1])]
        cx, cy, w, h = yi[ :4]*scale
        x1 = cx - w/2
        y1 = cy - h/2
        x2 = cx + w/2
        y2 = cy + h/2
        size = draw.textsize(cls, font)  #draw.textsize(cls, font)
        
        if y1 - size[1] >= 0:
            origin = np.array([x1, y1-size[1]])
        else:
            origin = np.array([x1, y1+1])
        
        for k in range(thickness):
            draw.rectangle([x1+k, y1+k, x2-k, y2-k], outline=tuple(color))
        # Draw Label
        draw.rectangle([tuple(origin), tuple(origin+size)], fill=tuple(color))
        draw.text(tuple(origin), cls, fill=(0, 0, 0), font=font) #draw.text(origin, cls, fill=(0, 0, 0), font=font)
        del draw
    return x


In [ ]:
#@title model { form-width: "20%" }   


class YoloLayer(torch.nn.Module):
    def __init__(self, anchors, stride, num_classes): # This layer is used at 3 different stages giving 3 different scales of an image, which is specific for yolo v.3
        super().__init__()
        self.anchors=torch.FloatTensor(anchors)/stride
        self.stride=stride
        self.num_classes=num_classes

    #here we extract boxes from the output of the network (output holding objectnes, class, ancor predictions)
    #we do this at 3 different stages giving us 3 different scales of an image, which is specific for yolo v.3
    def get_region_boxes(self, output, threshold):         
        if output.dim() == 3: output = output.unsqueeze(0)  
        device = output.device                             # torch.device(torch_device)
        anchors = self.anchors.to(device)
        
        B, c, H, W = output.size()
        A = len(anchors)
        C = self.num_classes
        Ad = B*A*H*W

        assert c == (C+5)*A

        #from output (in C+5 this 5 are x,y,w,h,box & Gx size is Ad) we make [(C+5) X Ad]
        output = output.view(B*A, C+5, H*W).transpose(0,1).contiguous().view(C+5, Ad)

        Gx = torch.arange(0, W).repeat(B*A, H).view(Ad).to(device)
        Gy = torch.arange(0, H).repeat(W, 1).t().repeat(B*A, 1).view(Ad).to(device)
        
        ix = torch.LongTensor(range(0,2)).to(device)
        Aw = anchors.index_select(1, ix[0]).repeat(1, B, H*W).view(Ad) #select from indexing dim 1, 0th element(-s) 
        Ah = anchors.index_select(1, ix[1]).repeat(1, B, H*W).view(Ad)

        # Gx and Gy represent left corner of the grid box, while output[0], output[1] are predicted anchor x,y
        # Aw, AH are real anchorbox dimensions relative to anchor and 
        # output[2], output[3] are predicted anchor box dimensions relative to anchor
        xs = torch.sigmoid(output[0]) + Gx   #we use sigmoid because predicted x,y are relative to Gx,Gy and
        ys = torch.sigmoid(output[1]) + Gy   #left top corner Gx,Gy + some fraction. Without sigmoid this breakes
        ws = torch.exp(output[2]) * Aw.detach()  # output[2] predicted Aw groud truth
        hs = torch.exp(output[3]) * Ah.detach()
        box_conf = torch.sigmoid(output[4])  # objectnes score is passed in sigmoid to interpret it as probability

        cls_conf = torch.nn.Softmax(dim=1)(output[5: C+5].transpose(0,1)).detach()
        cls_prob, cls_id = torch.max(cls_conf, 1)
        cls_prob = cls_prob.view(-1)
        cls_id = cls_id.view(-1)

        xs = xs.to('cpu')
        ys = ys.to('cpu')
        ws = ws.to('cpu')
        hs = hs.to('cpu')
        box_conf = box_conf.to('cpu') 
        cls_prob = cls_prob.to('cpu')
        cls_id = cls_id.to('cpu')


        boxes = [[] for i in range(B)]
        
        # first we filter output by objectnes score (confidence of having object(-s part) in receptive field)
        # and we ignore below treshold scored output boxes -- confidence tresholding NOT IoU !!!
        idx = torch.LongTensor(range(0,len(box_conf)))
        for i in idx[box_conf > threshold]:   #just take only those boxes with confidence score also aboeve some given score
            cx = xs[i]                        # NOT IoU threshold !!! do not mix this and IoU tresholds
            cy = ys[i]
            w = ws[i]
            h = hs[i]
            
            box = [cx/W, cy/H, w/W, h/H, box_conf[i], cls_prob[i], cls_id[i]]
            box = [i.item() for i in box]   
            
            batch = int(i.item()/(A*H*W))    # A*H*W is number of all anchors for each cell for one picture, so
            boxes[batch].append(box)         # each pictures bboxes will accumulate together

        return boxes

    
    def build_targets(self, pred_boxes, target, anchors, H, W): 
      #     all this is for: 
      #     1. create negative zero-masks to push down best predicted boxes overlaping with ground truth boxes!  
      #        As they are false concurents for ground truths. Call this modified non-maximum supression, NMS,
      #        because ground truth acts like maximum value element and we supress all best concurent boxes.     
      #     2. create positive target (mask=1 etc) masks from ground truths.
      #     3. Losses. MSE for box and object confidence (objectivnes) losses and CrossEntropy for class loss
      #        Here we push up energy to 1-s to positive 1-s target masks and implicitely also push down 
      #        false concurents to 0-s as are zero-masked using modified non-maximum supression, NMS.


        ignore_threshold = 0.5

        # Works faster on CPU than on GPU.
        dev = torch.device('cpu')
        pred_boxes = pred_boxes.to(dev)
        target = target.to(dev)
        anchors = anchors.to(dev)

        B = target.size(0)
        A, As = anchors.size()
        Ad = A*H*W
        
        box_mask   = torch.zeros(B, A, H, W)
        conf_mask  = torch.ones (B, A, H, W)
        cls_mask   = torch.zeros(B, A, H, W)
        t_boxes    = torch.zeros(4, B, A, H, W)
        t_conf     = torch.zeros(B, A, H, W)
        t_cls      = torch.zeros(B, A, H, W)

        # first modified/training specific non-max supresion NMS via zero-masking
        # NMS explanation https://learnopencv.com/non-maximum-suppression-theory-and-implementation-in-pytorch/
        # second one-masking of rescaled ground truths
        for b in range(B):                          #iterate in target target item length. target=(obj,classes,anchors) 
            p_box = pred_boxes[b*Ad: (b+1)*Ad].t()  #take predicted boxes per single image
            ious = torch.zeros(Ad)                  #prepare zero mask able to cover whole image
            box = target[b].view(-1,5)              #take each target boxes per single image

            # this part is for comparing (iou) targets vs predictions and to ignore predictions
            # under predefined treshold by setting their confidence masks to zeros
            for t in range(box.size(0)):    # iter through all boxes of one target image 
                if box[t][1] == 0:
                    break
                cx = box[t][0] * W          #take each target box metrics (of one image) 
                cy = box[t][1] * H          #and rescale it to predicted output size for IoU almost = NMS later etc.
                w  = box[t][2] * W
                h  = box[t][3] * H
                t_box = torch.FloatTensor([cx, cy, w, h]).repeat(Ad, 1).t() #single target box from target picture
                ious = torch.max(ious, SoftIoUnion(p_box, t_box))           #iou target vs predicted
            ignore = ious > ignore_threshold                           
            conf_mask[b][ignore.view(A, H, W)] = 0  #we eliminate predicted boxes, nearest neigbourhood(above treshold IoU) 
                                                    #of ground truth, because we backpropagate based on ground truth
                                                    #eliminated predicted boxes, are actually nearest to ground truth
                                                    #this is trainings type on non-max suppression NMS
            # here we create positive score=1 etc. masks grouped by most relevant anchors
            # theis is not a prediction part, everything is given and relevant anchors are just
            # anchors inside speciific interest cells
            for t in range(box.size(0)):            # again, iter through all boxes of one target image 
                if box[t][1] == 0:
                    break
                    
                cx = box[t][0] * W                  # again rescaling everything to the scale of predicted boxes
                cy = box[t][1] * H
                ci, cj = int(cx), int(cy) 
                w  = (box[t][2] * W).float()
                h  = (box[t][3] * H).float()      

                tmp_box = torch.FloatTensor([0, 0, w, h]).repeat(A, 1).t()#mask x,y. take w,h of each target box
                #then we make ground truth anchor box info comparable with 0,0,w,h shape of each target box:
                a_box = torch.cat((torch.zeros(A, As), anchors), 1).t() 

                #then we select groud truth boxes that correspond to ground truth anchors info(the most) best_a
                _, best_a = torch.max( SoftIoUnion(tmp_box, a_box), 0)  

                box_mask  [b][best_a][cj][ci] = 1 #we mask with anchor info verified groundtruth boxes with ones
                conf_mask [b][best_a][cj][ci] = 1
                cls_mask  [b][best_a][cj][ci] = 1
                
                t_boxes[0][b][best_a][cj][ci] = cx - ci #this is again to keep bbox center coordinates between 0-1
                t_boxes[1][b][best_a][cj][ci] = cy - cj #remember we did sigmoid for same purpose. This is because
                #we bounding box center coordinates are calculated relative to 'grid cell'-s left top corner
                #and this 'grid cell' is basically the receptive field of neuron. So we need just fractions.

                t_boxes[2][b][best_a][cj][ci] = math.log(w/anchors[best_a][0])
                t_boxes[3][b][best_a][cj][ci] = math.log(h/anchors[best_a][1])
                t_conf    [b][best_a][cj][ci] = 1
                t_cls     [b][best_a][cj][ci] = box[t][4]

        return box_mask, conf_mask, cls_mask, t_boxes, t_conf, t_cls


    def get_loss(self, output, target, return_single_value=True):
        device = output.device

        anchors = self.anchors.to(device)

        B, c, H, W = output.data.size()
        A = len(anchors)
        C = self.num_classes
        Ad = B*A*H*W

        output = output.view(B, A, (C+5), H, W)

        ix = torch.LongTensor(range(0,5)).to(device)
        p_boxes = output.index_select(2, ix[0:4]).view(B*A, -1, H*W).transpose(0,1).contiguous().view(4, Ad)
        
        p_boxes[0:2] = p_boxes[0:2].sigmoid()
        p_boxes[2:4] = p_boxes[2:4].exp()
        p_conf = output.index_select(2, ix[4]).view(B, A, H, W).sigmoid()

        Gx = torch.arange(0, W).repeat(B*A, H).view(Ad).to(device) #torch.arange(0, W)-->ascending from 0 to W 
        Gy = torch.arange(0, H).repeat(W, 1).t().repeat(B*A, 1).view(Ad).to(device)
        Aw = anchors.index_select(1, ix[0]).repeat(1, B*H*W).view(Ad)
        Ah = anchors.index_select(1, ix[1]).repeat(1, B*H*W).view(Ad)

        pred_boxes = torch.FloatTensor(4, Ad).to(device)
        pred_boxes[0] = p_boxes[0] + Gx
        pred_boxes[1] = p_boxes[1] + Gy
        pred_boxes[2] = p_boxes[2] * Aw
        pred_boxes[3] = p_boxes[3] * Ah 
        pred_boxes = pred_boxes.transpose(0,1).contiguous().view(-1,4)

        box_mask, conf_mask, cls_mask, t_boxes, t_conf, t_cls = \
            self.build_targets(pred_boxes.detach(), target.detach(), anchors.detach(), H, W)

        Gc = torch.arange(5, C+5).long().to(device)
        p_cls  = output.index_select(2, Gc)
        p_cls  = p_cls.view(B*A, C, H*W).transpose(1,2).contiguous().view(Ad, C)
        cls_mask = (cls_mask == 1)
        t_cls = t_cls[cls_mask].long().view(-1)
        cls_mask = cls_mask.view(-1, 1).repeat(1, C).to(device)
        p_cls = p_cls[cls_mask].view(-1, C)
        
        t_boxes = t_boxes.view(4, Ad).to(device)
        t_conf = t_conf.to(device)
        t_cls = t_cls.to(device)
        box_mask = box_mask.view(Ad).to(device)
        conf_mask = conf_mask.to(device)

        box_loss = torch.nn.MSELoss(reduction='sum')(p_boxes *box_mask, t_boxes *box_mask)
        conf_loss = torch.nn.MSELoss(reduction='sum')(p_conf *conf_mask, t_conf *conf_mask)
        cls_loss = torch.nn.CrossEntropyLoss(reduction='sum')(p_cls, t_cls) if p_cls.size(0) > 0 else 0
        loss = box_loss/2 + conf_loss + cls_loss

        if math.isnan(loss.item()):
            print(p_conf, t_conf)            
            raise ValueError('YoloLayer has isnan in loss')
        
        if return_single_value:
            return loss
        else:
            return [loss, box_loss/2, conf_loss, cls_loss]


class Yolov3Base(nn.Module, metaclass=ABCMeta):

    def __init__(self):
        super().__init__()

    @abstractmethod
    def get_loss_layers(self):
        return [self.yolo_0, self.yolo_1]

    def forward_backbone(self, x):
        return self.backbone(x)

    def forward(self, x):
        n, c, h, w = x.shape
        assert c==3 and h%32==0 and w%32==0, f"Tensor shape should be [bs, 3, x*32, y*32], but was {x.shape}"
        xb = self.forward_backbone(x)
        return self.forward_yolo(xb)

    def predict(self, x, threshold=0.25):
        self.eval()
        if x.dim() == 3:
            x = x.unsqueeze(0) 
        outputs = self.forward(x)
        return self.boxes_from_output(outputs, threshold)
    
    def boxes_from_output(self, outputs, threshold=0.25):
        boxes = [[] for j in range(outputs[0].size(0))]
        for i, layer in enumerate(self.get_loss_layers()):
            layer_boxes = layer.get_region_boxes(outputs[i], threshold=threshold)
            for j, layer_box in enumerate(layer_boxes):
                boxes[j] += layer_box
        return boxes

    def freeze_backbone(self, requires_grad=False):
        for p in self.backbone.parameters():
            p.requires_grad=requires_grad
    def unfreeze(self):
        for p in self.parameters():
            p.requires_grad = True
    def freeze_info(self, print_all=False):
        d = defaultdict(set)
        print("Layer: param.requires_grad")
        for name, param in self.named_parameters():
            if print_all:
                print(f"{name}: {param.requires_grad}")
            else:
                d[name.split('.')[0]].add(param.requires_grad)
        if not print_all:
            for k,v in d.items():
                print(k, ': ', v)        

    def load_backbone(self, h5_path):
        state_old = self.state_dict()
        state_new = torch.load(h5_path)

        skipped_layers = []
        for k in list(state_new.keys()):
            if state_old[k].shape != state_new[k].shape:
                skipped_layers.append(k)
                del state_new[k]

        return self.load_state_dict(state_new, strict=False), skipped_layers


class ConvBN(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=None):
        super().__init__()
        if padding is None:
            padding = (kernel_size - 1)//2
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels, momentum=0.01)
        self.relu = nn.LeakyReLU(0.1, inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class Upsample(nn.Module):
    def __init__(self, stride=2):
        super().__init__()
        self.stride = stride
    def forward(self, x):
        assert(x.data.dim() == 4)
        return nn.Upsample(scale_factor=self.stride, mode='nearest')(x)

class Yolov3UpsamplePrep(nn.Module):
    def __init__(self, filters_list, in_filters, out_filters):
        super().__init__()
        self.branch = nn.ModuleList([
                        ConvBN(in_filters, filters_list[0], 1),
                        ConvBN(filters_list[0], filters_list[1], kernel_size=3),
                        ConvBN(filters_list[1], filters_list[0], kernel_size=1),
                        ConvBN(filters_list[0], filters_list[1], kernel_size=3),
                        ConvBN(filters_list[1], filters_list[0], kernel_size=1),])
        self.for_yolo = nn.ModuleList([
                        ConvBN(filters_list[0], filters_list[1], kernel_size=3),
                        nn.Conv2d(filters_list[1], out_filters, kernel_size=1, stride=1,
                                   padding=0, bias=True)])

    def forward(self, x):
        for m in self.branch: x = m(x)
        branch_out = x
        for m in self.for_yolo: x = m(x)
        return branch_out, x

class DarknetBlock(nn.Module):
    def __init__(self, ch_in):
        super().__init__()
        ch_hid = ch_in//2
        self.conv1 = ConvBN(ch_in, ch_hid, kernel_size=1, stride=1, padding=0)
        self.conv2 = ConvBN(ch_hid, ch_in, kernel_size=3, stride=1, padding=1)

    def forward(self, x): return self.conv2(self.conv1(x)) + x


class Darknet(nn.Module):
    def __init__(self, num_blocks, start_nf=32):
        super().__init__()
        nf = start_nf
        self.base = ConvBN(3, nf, kernel_size=3, stride=1)
        self.layers = []
        for i, nb in enumerate(num_blocks):
            dn_layer = self.make_group_layer(nf, nb, stride=2)
            self.add_module(f"darknet_{i}", dn_layer)
            self.layers.append(dn_layer)
            nf *= 2

    def make_group_layer(self, ch_in, num_blocks, stride=2):
        layers = [ConvBN(ch_in, ch_in*2, stride=stride)]
        for i in range(num_blocks): layers.append(DarknetBlock(ch_in*2))
        return nn.Sequential(*layers)

    def forward(self, x):
        y = [self.base(x)]
        for l in self.layers:
            y.append(l(y[-1]))
        return y


class Yolov3(Yolov3Base):
    def __init__(self, num_classes=80):
        super().__init__()
        self.backbone = Darknet([1,2,8,8,4])
        
        A = 3
        self.yolo_0_pre = Yolov3UpsamplePrep([512, 1024], 1024, A*(num_classes+5))
        self.yolo_0 = YoloLayer(anchors=[(116.,  90.), (156., 198.), (373., 326.)], stride=32, num_classes=num_classes)

        self.yolo_1_c = ConvBN(512, 256, 1)
        self.yolo_1_prep = Yolov3UpsamplePrep([256, 512], 512+256, A*(num_classes+5))
        self.yolo_1 = YoloLayer(anchors=[(30., 61.), (62., 45.), (59., 119.)], stride=16, num_classes=num_classes)

        self.yolo_2_c = ConvBN(256, 128, 1)
        self.yolo_2_prep = Yolov3UpsamplePrep([128, 256], 256+128, A*(num_classes+5))
        self.yolo_2 = YoloLayer(anchors=[(10., 13.), (16., 30.), (33., 23.)], stride=8, num_classes=num_classes)

    def get_loss_layers(self):
        return [self.yolo_0, self.yolo_1, self.yolo_2]

    def forward_yolo(self, xb):
        x, y0 = self.yolo_0_pre(xb[-1])

        x = self.yolo_1_c(x)
        x = nn.Upsample(scale_factor=2, mode='nearest')(x)
        x = torch.cat([x, xb[-2]], 1)
        x, y1 = self.yolo_1_prep(x)

        x = self.yolo_2_c(x)
        x = nn.Upsample(scale_factor=2, mode='nearest')(x)
        x = torch.cat([x, xb[-3]], 1)
        x, y2 = self.yolo_2_prep(x)
        
        return [y0, y1, y2]


#------------------------------- here starts tiny yolo 


class MaxPoolStride1(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = F.max_pool2d(F.pad(x, (0,1,0,1), mode='replicate'), 2, stride=1)
        return x

class Yolov3TinyBackbone(nn.Module):
    def __init__(self, input_channels=3):
        super().__init__()
        self.layers =  nn.Sequential(OrderedDict([
            ('0_convbatch',     ConvBN(input_channels, 16, 3, 1, 1)),
            ('1_max',           nn.MaxPool2d(2, 2)),
            ('2_convbatch',     ConvBN(16, 32, 3, 1, 1)),
            ('3_max',           nn.MaxPool2d(2, 2)),
            ('4_convbatch',     ConvBN(32, 64, 3, 1, 1)),
            ('5_max',           nn.MaxPool2d(2, 2)),
            ('6_convbatch',     ConvBN(64, 128, 3, 1, 1)),
            ('7_max',           nn.MaxPool2d(2, 2)),
            ('8_convbatch',     ConvBN(128, 256, 3, 1, 1)),
            ('9_max',           nn.MaxPool2d(2, 2)),
            ('10_convbatch',    ConvBN(256, 512, 3, 1, 1)),
            ('11_max',          MaxPoolStride1()),
            ('12_convbatch',    ConvBN(512, 1024, 3, 1, 1)),
            ('13_convbatch',    ConvBN(1024, 256, 1, 1, 0))]))
    
    def forward(self, x):
        idx=9
        x0 = self.layers[ :idx](x)
        x1 = self.layers[idx: ](x0)
        return x0, x1


class Yolov3Tiny(Yolov3Base):

    def __init__(self, num_classes, use_wrong_anchors=False):
        super().__init__()

        self.num_classes = num_classes
        self.return_out_boxes = False
        self.skip_backbone = False

        self.backbone = Yolov3TinyBackbone()

        Ar = 3
        self.yolo_0_pre = nn.Sequential(OrderedDict([
            ('14_convbatch',    ConvBN(256, 512, 3, 1, 1)),
            ('15_conv',         nn.Conv2d(512, (num_classes+5)*Ar, 1, 1, 0))]))
        
        anchors0=[(81.,82.), (135.,169.), (344.,319.)]
        self.yolo_0 = YoloLayer(anchors=anchors0, stride=32, num_classes=num_classes)

        self.up_1 = nn.Sequential(OrderedDict([
            ('17_convbatch',    ConvBN(256, 128, 1, 1, 0)),
            ('18_upsample',     Upsample(2))]))

        self.yolo_1_pre = nn.Sequential(OrderedDict([
            ('19_convbatch',    ConvBN(128+256, 256, 3, 1, 1)),
            ('20_conv',         nn.Conv2d(256, (num_classes+5)*Ar, 1, 1, 0))]))

        if use_wrong_anchors:
            anchors1 = [(23.,27.),  (37.,58.),  (81.,82.)]
        else: 
            anchors1 = [(10.,14.),  (23.,27.),  (37.,58.)]

        self.yolo_1 = YoloLayer(anchors=anchors1, stride=16, num_classes=num_classes)

    def get_loss_layers(self):
        return [self.yolo_0, self.yolo_1]

    def forward_yolo(self, xb):
        x0, x1 = xb[0], xb[1]
        y0 = self.yolo_0_pre(x1)

        x_up = self.up_1(x1)
        x_up = torch.cat((x_up, x0), 1)
        y1 = self.yolo_1_pre(x_up)
        return [y0, y1]


In [ ]:
#@title set up for training { form-width: "20%" }

path = '/content/' 

train_set = VOCDataset(path, input_shape=(320, 320), mode='train', dataset='voc')
train_loaded = DataLoader(train_set, batch_size=1, shuffle=False, drop_last=True)

model = Yolov3Tiny(num_classes=len(train_set.classes))         

x, y = train_set.__getitem__(10)
PlotSample(x, y, dataset='voc')

In [ ]:
#@title training { form-width: "20%" }

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device) 
optimizer = torch.optim.Adam(model.parameters()) 

def YoloLoss(model, pred, target):
    layers=model.get_loss_layers()
    loss=0.
    for i, layer in enumerate(layers):
        loss += layer.get_loss(pred[i], target, return_single_value=True)
    return loss


def Train(model, criterion, optimizer, train_loaded, epochs=5):
    train_losses=[]
    model.train()
    for epoch in tqdm(range(epochs)):
        losses=0.
        for x, y in tqdm(train_loaded):
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(model, pred, y)
            loss.backward()
            optimizer.step()
            losses+=loss
        train_losses.append(losses/len(train_loaded))
        print('Epoch {}/{} --- Loss: {:.4f}'.format(epoch+1, epochs, train_losses[-1]))
    return train_losses

loss = Train(model, YoloLoss, optimizer, train_loaded, epochs=20) # epochs=20 gives good result

In [ ]:
#@title plotting the loss { form-width: "20%" }

print(loss[0].item())

items = [l.item() for l in loss]
plt.plot(items)
plt.title('Loss');

In [ ]:
#@title save and load trained { form-width: "20%" }

torch.save(model.state_dict(), 'yolov3_tiny.pth') #torch.save(model.state_dict(), 'yolov3_tiny.h5')

model = Yolov3Tiny(num_classes=20)    # voc has 20 and coco 80 clases
model.load_state_dict(torch.load('/content/yolov3_tiny.pth')) # model.load_state_dict(torch.load('/content/yolov3_tiny.h5'))

<All keys matched successfully>

In [ ]:
#@title detect { form-width: "20%" }

image = '/content/JPEGImages/000200.jpg'
model.to('cpu') # or vice versa put image to GPU but you'll need to use PIL first

%time
PictureDetection(model, image, size=(320,320), dataset='voc')

In [ ]:
#@title did not like results here more training { form-width: "20%" }

# optimizer = torch.optim.Adam(model.parameters()) 
# model.to(device) 

# loss = Train(model, YoloLoss, optimizer, train_loaded, epochs=1)